In [1]:
import numpy as np

In [2]:
def givens_rotations(a, b, c, d):
    """Calculates the angles needed for a Givens rotation to out put the state with amplitudes a,b,c and d

    Args:
        - a,b,c,d (float): real numbers which represent the amplitude of the relevant basis states (see problem statement). Assume they are normalized.

    Returns:
        - (list(float)): a list of real numbers ranging in the intervals provided in the challenge statement, which represent the angles in the Givens rotations,
        in order, that must be applied.
    """

    # QHACK #
    # Here we let Z = theta_3/2, Y = theta_2/2 and X = theta_1/2
    Z = np.arctan(-d/a)
    Y = np.arctan(- c / b)
    X = np.arctan(-(c * np.sin(Z)) / (d * np.sin(Y)))
    return X*2, Y*2, Z*2

In [3]:
inputs = "0.5,0.5,0.5,0.5".split(",")
theta_1, theta_2, theta_3 = givens_rotations(
    float(inputs[0]), float(inputs[1]), float(inputs[2]), float(inputs[3])
)
print(*[theta_1, theta_2, theta_3], sep=",")

-1.5707963267948966,-1.5707963267948966,-1.5707963267948966


In [4]:
import random
import pennylane as qml

dev = qml.device("default.qubit", wires=[0, 1, 2, 3, 4, 5])

@qml.qnode(dev)
def circuit(t1, t2, t3):
    # prepares the reference state |100>
    qml.BasisState(np.array([1, 1, 0, 0, 0, 0]), wires=[0, 1, 2, 3, 4, 5])
    # applies the single excitations
    qml.DoubleExcitation(t1, wires=[0, 1, 2, 3])
    qml.DoubleExcitation(t2, wires=[2, 3, 4, 5])
    qml.ctrl(qml.SingleExcitation, control=0)(t3, wires=[1, 3])
    return qml.state()

def new_experiment(params=None, print_outputs=False):
    if params is not None:
        t1, t2, t3 = params[0], params[1], params[2]
    else:
        t1, t2, t3 = (random.random()*2*np.pi - np.pi)/2, (random.random()*2*np.pi - np.pi)/2, (random.random()*2*np.pi - np.pi)/2
    tensor_state = circuit(t1, t2, t3).reshape(2, 2, 2, 2, 2, 2)
    
    a = tensor_state[1, 1, 0, 0, 0, 0].real
    b = tensor_state[0, 0, 1, 1, 0, 0].real
    c = tensor_state[0, 0, 0, 0, 1, 1].real
    d = tensor_state[1, 0, 0, 1, 0, 0].real
    
    Z = np.arctan(-d / a)
    Y = np.arctan(-c / b)
    X = np.arctan(-(c*np.sin(Z))/(d*np.sin(Y)))
    
    if print_outputs:
        print(f"theta_1, theta_2, theta_3 = {t1},{t2},{t3}")
        print()
        print("Amplitude of state |110000> = ", tensor_state[1, 1, 0, 0, 0, 0].real)
        print("Amplitude of state |001100> = ", tensor_state[0, 0, 1, 1, 0, 0].real)
        print("Amplitude of state |000011> = ", tensor_state[0, 0, 0, 0, 1, 1].real)
        print("Amplitude of state |100100> = ", tensor_state[1, 0, 0, 1, 0, 0].real)
        print()
        print("a", a)
        print("b", b)
        print("c", c)
        print("d", d)
        print()
        print(f"Real theta_1    : {t1}")
        print(f"Estimate theta_1: {X*2}")
        print(f"Real theta_2    : {t2}")
        print(f"Estimate theta_2: {Y*2}")
        print(f"Real theta_3    : {t3}")
        print(f"Estimate theta_3: {Z*2}")
        print()
    
    if np.round(X*2, 6) == np.round(t1, 6) and np.round(Y*2, 6) == np.round(t2, 6) and np.round(Z*2, 6) == np.round(t3, 6):
        print(f"Found the angles {X*2},{Y*2},{Z*2}!!!!")
    else:
        print("Couldn't find the angles : (")
        print_outputs = True

In [5]:
new_experiment(print_outputs=True)

theta_1, theta_2, theta_3 = -0.36486066102021986,-0.3581602934776127,-0.08614443586073484

Amplitude of state |110000> =  0.9824936127035108
Amplitude of state |001100> =  0.17851882961867488
Amplitude of state |000011> =  0.03231536705879838
Amplitude of state |100100> =  0.04234436822338739

a 0.9824936127035108
b 0.17851882961867488
c 0.03231536705879838
d 0.04234436822338739

Real theta_1    : -0.36486066102021986
Estimate theta_1: -0.36486066102021975
Real theta_2    : -0.3581602934776127
Estimate theta_2: -0.3581602934776127
Real theta_3    : -0.08614443586073484
Estimate theta_3: -0.08614443586073481

Found the angles -0.36486066102021975,-0.3581602934776127,-0.08614443586073481!!!!
